# Phase 1 Notebook: `provider.py`

Target file: `execution/langgraph/provider.py`

Purpose: isolate model-provider behavior behind one interface so orchestration logic remains provider-agnostic.

## 3-Layer Context

- Layer 1 (Directive): Phase 1 asks for framework literacy plus reliable execution boundaries.
- Layer 2 (Orchestration): `graph.py` depends on provider abstraction, not direct SDK calls.
- Layer 3 (Execution): provider adapters transform message schema into deterministic SDK requests.

In [ ]:
from pathlib import Path
import sys
import os

def bootstrap_repo_root() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd, *cwd.parents, Path('/home/nir/dev/agent_phase0')]
    for candidate in candidates:
        if (candidate / 'execution' / 'langgraph' / 'provider.py').exists():
            if str(candidate) not in sys.path:
                sys.path.insert(0, str(candidate))
            return candidate
    raise RuntimeError('Could not locate repo root for provider notebook.')

repo_root = bootstrap_repo_root()
from dotenv import load_dotenv
load_dotenv(repo_root / ".env")
print('P1_PROVIDER =', os.getenv('P1_PROVIDER', 'ollama'))
print('repo_root =', repo_root)
print('kernel_python =', sys.executable)
if '/.venv/' not in sys.executable.replace('\\', '/'):
    print('WARNING: kernel is not the project .venv interpreter.')

## P0 vs P1 (Provider Boundary)

| Concern | Phase 0 (`llm_provider.py`) | Phase 1 (`provider.py`) | Why this matters |
|---|---|---|---|
| Provider coupling | Single Groq path | OpenAI adapter + Groq adapter + selector | Switch providers without touching orchestration logic |
| Interface | Concrete class | `ChatProvider` protocol | Easier mocking in tests and notebooks |
| Failure handling | Direct call path | Fallback strategy (`build_provider`) | Better operational resilience |

## Why this block exists: inspect adapter contracts

We inspect source to verify provider adapters expose one consistent `generate(messages)` API used by the graph.

In [ ]:
import inspect
import execution.langgraph.provider as provider_mod

print(inspect.getsource(provider_mod.ChatProvider))
print(inspect.getsource(provider_mod.OpenAIChatProvider))
print(inspect.getsource(provider_mod.GroqChatProvider))
print(inspect.getsource(provider_mod.build_provider))

## Why this block exists: local deterministic mock

In P1 we test orchestration without network calls by using a scripted provider object that satisfies the same protocol.

In [ ]:
import json

class ScriptedProvider:
    def __init__(self, responses):
        self.responses = [json.dumps(item) for item in responses]
        self.index = 0

    def generate(self, messages):
        if self.index < len(self.responses):
            value = self.responses[self.index]
            self.index += 1
            return value
        return self.responses[-1]

mock = ScriptedProvider([{'action': 'finish', 'answer': 'ok'}])
mock.generate([])

In [ ]:
# Assertion: protocol-compatible mock must expose generate(messages) -> str
result = mock.generate([{'role': 'user', 'content': 'hi'}])
assert isinstance(result, str)
print('provider mock assertion passed')

## Takeaways

- Provider abstraction decouples orchestration from vendor SDKs.
- P1 keeps deterministic testing possible via scripted providers.
- This file is a core seam for future provider/cost policy routing.
- Next notebook: `execution/notebooks/p1_policy.ipynb`.